# Preparing DENUE's dataset for matching

# General information

VERSION 1.0
Original release

VERSION 1.1
This is a modification required to process `'/scratch/public/jpvasquez/MNCs_informality/Raw_data/DENUE/output/denue_version_both.csv'` rather than the original file. The latter can be process with this file removing some cells and the hashes (#). 

The objective of this code is divided in these main objectives: 

- Recode the geographical zones. 
- Remove the firms that lack geographical location (this happened in the dataset construction of the Raw data). 
- Filter the firms which meet the following criteria: 
    - The firm has at least one business name associated (razon social). 
    - The firm is a big company (more than 50 employees). 
- Reshape the dataset such that each observation corresponds to a single possible name of a firm in its location. 
- Remove 'stopwords'. 
- Save the dataset. 

Note that we aren't cleaning the data because this was done in the data set's creation code in `'/scratch/public/jpvasquez/MNCs_informality/Raw_data/DENUE/code/build.do'`. 

# Input files
1. **denue:** `'/scratch/public/jpvasquez/MNCs_informality/Raw_data/DENUE/output/denue_version_both.csv'` This file has geographical location variables, number of workers, ORBIS and DENUE's firm keys, generic firm names and business names (razon social). 
2. **denue_geo_corrected:** `'/scratch/public/jpvasquez/MNCs_informality/Intermediate_data/data/denue_municipalities_corrected.csv'` This file contains the geographical zones of firms with their proper coding, which procedure is documented in in `'/scratch/public/jpvasquez/MNCs_informality/Intermediate_data/data/cleaning_datasets_orbis_denue.md'`. 

In [1]:
denue_file = '/scratch/public/jpvasquez/MNCs_informality/Raw_data/DENUE/output/denue_version_both.csv'
denue_geo_corrected_file = '/scratch/public/jpvasquez/MNCs_informality/Intermediate_data/data/denue_municipalities_corrected.csv'

# Output files
1. **denue_final:** `'/scratch/public/jpvasquez/MNCs_informality/Intermediate_data/output/denue_final.csv'` This file contains a dataset where each row represents a firm with one of their names associated, also, the number of workers in that firm, entity, municipality and DENUE's key. 
2. **denue_final_names:** `'/scratch/public/jpvasquez/MNCs_informality/Intermediate_data/output/denue_final_names.csv'` This file contains a data set with each unique `llave_denue` with all their names associated to it. 

In [2]:
denue_final = '/scratch/public/jpvasquez/MNCs_informality/Intermediate_data/output/denue_final.csv'
denue_names_file = '/scratch/public/jpvasquez/MNCs_informality/Intermediate_data/output/denue_names.csv'

# Packages
These are the needed packages to run this code. In case, the machine you're running this in doesn't have any of these packages, run this code: 

`!pip install package_name`

**Pandas** is the package which handles importing, wrangling, cleaning and doing everything with the data. 

In [3]:
import pandas as pd

# Importing the data

In [4]:
denue = pd.read_csv(denue_file, engine = 'python')
denue_geo_corrected = pd.read_csv(denue_geo_corrected_file, sep = ';')

# Recode geographical zones
Using merge on `entidad` and `municipalidad`, the correct names of the geographical zones are associated according to the assigned codes `entidad_corrected` and `municipality_corrected` in `denue_geo_corrected`. 

# Removing the firms that lack geographical zone
When merging the data sets, the option *inner* is used such that the firms that remain in `denue` are the ones that have an `entidad` value in `denue` which is in the set of possible values of `entidad` in `denue_geo_corrected`. In case the data set is fixed, either *inner* or *left* can be used. 

In [5]:
denue = denue.merge(denue_geo_corrected, how = 'inner', 
                    left_on = ['entidad', 'municipio'], 
                    right_on = ['entidad', 'municipio'])

# Remove the geographical information which isn't needed

In [6]:
#denue = denue.drop(columns = ['entidad', 'municipio', 
#                              'postal_code', 'localidad', 
#                              'ageb', 'manzana', 
#                              'latitud', 'longitud'])
denue = denue.drop(columns = ['entidad', 'municipio', 
                              'postal_code', 'localidad'])

# Filter the firms
## The firm has at least one business name associated (razon social). 
A dummy variable is created in order to match the corresponding condition. If the firm has one of their business names not equal to a missing value, then the firm is elegible of type 1. 

In [7]:
denue['elegible_1'] = 0
denue.loc[(denue['razon_social_1'].notna()) | 
          (denue['razon_social_2'].notna()) | 
          (denue['razon_social_3'].notna()) | 
          (denue['razon_social_4'].notna()) | 
          (denue['razon_social_5'].notna()) | 
          (denue['razon_social_6'].notna()) | 
          (denue['razon_social_7'].notna()) | 
          (denue['razon_social_8'].notna())]['elegible_1'] = 1

/usr/local/linux/anaconda3.7/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


## The firm is a big company (more than 50 employees). 
A dummy variable is created in order to match the corresponding condition. If the firm has 51 to 100 employees, 101 to 250 or 251 or more, then the firm is elegible of type 2. 

In [8]:
denue['elegible_2'] = denue['n_workers'].map({'1) 0 a 5 personas': 0, # write 1 in you want a specific category
                                              '2) 6 a 10 personas': 0,  # in the dataset, else, write 0
                                              '3) 11 a 30 personas': 0, 
                                              '4) 31 a 50 personas': 0, 
                                              '5) 51 a 100 personas': 1, 
                                              '6) 101 a 250 personas': 1, 
                                              '7) 251 y más personas': 1})

## Select the observations 
We select the observations that meet the criteria, also, we reset the index to keep it aligned with the number of rows, the latter procedure we'll be done multiple times. Finally, we drop the `elegible` variables. Remember, these are dummy variables we created, so, you can write `True` or `False`, `1` or `0` to include or exclude them, respectively. 

In [9]:
denue = (denue[(denue['elegible_1'] == 1) | (denue['elegible_2'] == 1)]
         .reset_index())

# Preparing the data for a reshape

## Drop the unnecessary variables

In [10]:
denue = denue.drop(columns = 'elegible_1')

## Rename the columns

In [11]:
denue = denue.rename(columns = {'name_1': 'firm1', 
                                'name_2': 'firm2', 
                                'name_3': 'firm3', 
                                'name_4': 'firm4', 
                                'name_5': 'firm5', 
                                'name_6': 'firm6', 
                                'name_7': 'firm7', 
                                'razon_social_1': 'firm8', 
                                'razon_social_2': 'firm9', 
                                'razon_social_3': 'firm10', 
                                'razon_social_4': 'firm11', 
                                'razon_social_5': 'firm12', 
                                'razon_social_6': 'firm13', 
                                'razon_social_7': 'firm14', 
                                'name_8': 'firm15', 
                                'razon_social_8': 'firm16', 
                                'entidad_corrected': 'entidad', 
                                'municipio_corrected': 'municipio'})

# Drop duplicates from data set C
Now that we've got two data sets appended, there might be some duplicates with the same *llave_denue*. So, we keep the ones in data set d. 

In [12]:
denue = (denue.sort_values(by = 'data_base', ascending = False)
         .drop_duplicates(subset = ['llave_denue'], 
                          keep = 'first')
         .drop(columns = ['data_base']))

# Reshape the data set
The data set is reshaped in order that every row represents a single possible name of a single firm with their respective location. Also, note that not every `name` or `razon_social` variable has necessarily a name in it, there can be just one name reported. This will give missing values or NAs in our dataset, so we drop the missing values. Finally, we reset the index. 

In [13]:
denue = (pd.wide_to_long(denue, stubnames = 'firm', i = 'index', j='n')
         .dropna()
        )

# Clean the data base

## Replacing _ with spaces
First, we remove everything that's not a letter or a white space f in every `firm`. Then, removing the _ can leave multiple spaces, so we make sure that between every word there's only one space with `strip` method.  

In [14]:
denue['firm'] = (denue['firm'].str.replace('[^\w\s]','')
                 .str.replace('_',' ')
                 .str.strip())

## Getting the unique names for each `llave_denue`
We select the key column `llave_denue` and the firm names `firm`, then, we drop possible duplicates and reset the index. 

In [15]:
denue_names = (denue[['llave_denue', 'firm']]
               .copy()
               .drop_duplicates()
               .reset_index(drop = True))

## Create dummy column to count unique firm names by `llave_denue`

In [16]:
denue_names['n'] = (denue_names
                    .groupby(['llave_denue'])
                    .cumcount())

## Pseudo reshaping from long to wide
This is done in order to get an observation with a unique `llave_denue` with all the possible `firm` names associated with it. 

In [17]:
denue_names = denue_names.set_index(['llave_denue', 'n']).unstack()

# Overwrite MultiIndex with desired column names

In [18]:
denue_names.columns = ['firm1', 
                       'firm2', 
                       'firm3', 
                       'firm4', 
                       'firm5', 
                       'firm6', 
                       'firm7', 
                       'firm8', 
                       'firm9', 
                       'firm10', 
                       'firm11']

## Save `denue_names`

In [19]:
denue_names.to_csv(denue_names_file)

## Removing stopwords
There are multiple words that don't add more information or quality to our matching algorithms. We can assure this because all the firms have the same location: México, and we don't care about the company's structure in name similarity. Also, by looking manually in the data set, we detected common words that could qualify as stopwords and made a list with them. Then, for each possible firm name, we create a vector without the stopwords listed and joined them with spaces again. 

In [20]:
remove_words = ['de', 'a', 's', 'l', 
                'r', 'sade', 'v', 'c', 
                'b', 'sa', 'cv', 'sab', 
                'mexicana', 'mexicano', 'limitada', 'rl', 
                'mexico', 'latinoamerica', 'srl', 'mejico', 
                'via', 'its', 'funds', 'y', 
                'sapi', 'enr', 'sofom', 'mxico', 
                'latin', 'america', 'internacional', 'mexicanos', 
                'mexicanas', 'mex', 'er']
denue['firm'] = (denue['firm'].apply(lambda x: ' '
                                     .join([word for word in x.split() 
                                            if word not in (remove_words)])))

## Making sure there aren't duplicates
We drop the duplicates and reset the index, notice that there are many of them after remove punctuation, accents and multiple spaces. 

In [21]:
denue = (denue.drop_duplicates()
         .reset_index(drop = True))

In [22]:
denue

,municipio,n_workers,elegible_2,llave_denue,entidad,firm
0,toluca,7) 251 y más personas,1.0,2421131,mexico,abarrotes del centro
1,ahome,5) 51 a 100 personas,1.0,3776371,sinaloa,secretaria agricultura ganade
2,ahome,5) 51 a 100 personas,1.0,3778355,sinaloa,iglesia jesucristo los san
3,ahome,5) 51 a 100 personas,1.0,3778309,sinaloa,iglesia apostolica la fe en
4,ahome,6) 101 a 250 personas,1.0,3777973,sinaloa,japama planta ingeniero jose her
...,...,...,...,...,...,...
334984,centro,6) 101 a 250 personas,1.0,6399538,tabasco,fideicomisof1596 fideicomisof159
334985,paracho,5) 51 a 100 personas,1.0,2566782,michoacan_de_ocampo,preciliano
334986,cuauhtemoc,6) 101 a 250 personas,1.0,6313867,ciudad_de_mexico,alfaomega grupo editorial d
334987,cuauhtemoc,5) 51 a 100 personas,1.0,1005620,ciudad_de_mexico,caf restaurantes del centro d


# Save the data set
We save the data set to a Comma Separated Values file and we order the columns in our preferred order by naming them one by one. 

In [23]:
denue[['llave_denue', 'n_workers','entidad', 'municipio', 'firm', 'elegible_2']].to_csv(denue_final, index = False)